<a href="https://colab.research.google.com/github/Richsrivastava/End-End_Data_Science_Workflows/blob/main/Dense_Geographic_Cluster_Estimate_FindInfected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext cudf.pandas
import pandas as pd
import cuml

import cupy as cp

In [3]:
# load google drive folder to check if data file is available
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks"

Mounted at /content/drive
 clinics.csv
 code_guide.csv
'Copy of Data Science Find Infected.ipynb'
'Copy of replytors RunResearch.ipynb'
 Dense_Geographic_Cluster_Estimate_FindInfected.ipynb
 HelloWorld.ipynb
 HF_My_First_Agent.ipynb
 hospitals.csv
 HuggingFaceFirstAgent.ipynb
 MultiAgent.ipynb
 MultiAgentThatWorks.ipynb
 Pandas101.ipynb
'replytors RunResearch.ipynb'
 TRialDuckDuckGoSEarch.ipynb
 Untitled
 Untitled0.ipynb
'Untitled (1)'
 Untitled1.ipynb
 week1.csv
 week2.csv
 week3.csv


In [4]:
# load data about week 1 of the outbreak into a cuDF-accelerated pandas DataFrame. Only need the 'lat', 'long', and 'infected' columns.
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/week1.csv", usecols=["lat", "long", "infected"])
df

,lat,long,infected
0,54.522510,-1.571896,False
1,54.554030,-1.524968,False
2,54.552486,-1.435203,False
3,54.537189,-1.566215,False
4,54.528212,-1.588462,False
...,...,...,...
58479889,51.634416,-2.925863,False
58479890,51.556972,-3.036290,False
58479891,51.588992,-2.921915,False
58479892,51.590974,-2.954539,False


In [5]:
#Make new DataFrame that contains only the infected members of the population
infected_df = df[df['infected']== 1]
infected_df = infected_df.reset_index(drop=True)
infected_df.head()
#infected_df.shape

,lat,long,infected
0,54.472766,-1.654932,True
1,54.529717,-1.667143,True
2,54.512986,-1.589866,True
3,54.522322,-1.380694,True
4,54.541660,-1.613490,True


In [6]:
# lat/long to OSGB36 grid coordinates converter

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.

    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians

    Output:
    (northing, easting)
    '''

    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian

    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)

    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff -
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) -
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [7]:
# Use above converter to create grid coordinate values stored in northing and easting columns of the infected_df
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(cupy_lat, cupy_long,input_degrees=True )
infected_df

,lat,long,infected,northing,easting
0,54.472766,-1.654932,True,508670.060234,422359.759523
1,54.529717,-1.667143,True,515002.666798,421538.547038
2,54.512986,-1.589866,True,513167.535850,426549.874086
3,54.522322,-1.380694,True,514305.280055,440081.234798
4,54.541660,-1.613490,True,516349.132042,425003.005560
...,...,...,...,...,...
18143,52.428347,-3.322932,True,282016.338253,310060.098268
18144,52.415895,-3.263942,True,280559.681381,314046.146547
18145,52.539934,-3.617128,True,294832.815870,290338.202721
18146,52.435490,-3.597263,True,283187.465568,291428.293249


In [8]:
# Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member.
dbscan = cuml.DBSCAN(eps=2000, min_samples=25)
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df

,lat,long,infected,northing,easting,cluster
0,54.472766,-1.654932,True,508670.060234,422359.759523,-1
1,54.529717,-1.667143,True,515002.666798,421538.547038,-1
2,54.512986,-1.589866,True,513167.535850,426549.874086,-1
3,54.522322,-1.380694,True,514305.280055,440081.234798,-1
4,54.541660,-1.613490,True,516349.132042,425003.005560,-1
...,...,...,...,...,...,...
18143,52.428347,-3.322932,True,282016.338253,310060.098268,-1
18144,52.415895,-3.263942,True,280559.681381,314046.146547,-1
18145,52.539934,-3.617128,True,294832.815870,290338.202721,-1
18146,52.435490,-3.597263,True,283187.465568,291428.293249,-1


In [9]:
# Find centroid of each cluster
centroids_df = infected_df[['northing', 'easting', 'cluster']].groupby('cluster')[['northing', 'easting']].mean()
centroids_df

,northing,easting
cluster,,
-1,378085.504251,401877.070477
0,397661.052147,371410.022807
1,436475.467158,332980.455514
2,347062.237166,389386.821165
3,359668.638420,379638.020073
4,391630.079963,431158.142881
5,386471.292123,426559.091880
6,434970.334950,406985.282976
7,412772.647531,410069.665645


In [10]:
# find the number of people in each cluster by counting the number of appearances of eah cluster's label in the column produced by DBSCAN
centroids_df['count'] = infected_df['cluster'].value_counts()
centroids_df

,northing,easting,count
cluster,,,
-1,378085.504251,401877.070477,8449
0,397661.052147,371410.022807,8638
1,436475.467158,332980.455514,68
2,347062.237166,389386.821165,403
3,359668.638420,379638.020073,25
4,391630.079963,431158.142881,66
5,386471.292123,426559.091880,43
6,434970.334950,406985.282976,27
7,412772.647531,410069.665645,39


In [11]:
# Find the Centroid of the Cluster with the Most Members and write the answer to my_assessment/question_1.json.

centroids_df.loc[centroids_df['count'].idxmax()].to_json('question_1.json')


/usr/local/lib/python3.11/dist-packages/cudf/io/json.py:423: UserWarning: Using CPU via Pandas to write JSON dataset
  warnings.warn("Using CPU via Pandas to write JSON dataset")


In [12]:
#check submission
!cat question_1.json

{"northing":397661.0521473512,"easting":371410.0228066574,"count":8638.0}